<a href="https://colab.research.google.com/github/yalonzo/analytics/blob/main/get_weather_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APIs Metereológicas

## METEOSTAT

In [1]:
!pip install meteostat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily

import plotly.express as px

In [3]:
## configuración del periodo de tiempo
start = datetime(2022, 1, 1)
end = datetime(2022, 12, 31)

## creación de un punto de referencia
#vancouver = Point(49.2497, -123.1193, 70) ##lat, long, alt
lima = Point(-12.0891, -76.9988)

## obtención de la data diaria
data = Daily(lima, start, end)
df_meteostat = data.fetch()
df_meteostat

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2022-01-01,21.2,19.0,24.0,NaN,NaN,193.0,12.9,NaN,1012.6,NaN
2022-01-02,21.1,19.9,23.0,NaN,NaN,172.0,15.8,NaN,1013.8,NaN
2022-01-03,21.3,18.0,24.0,NaN,NaN,189.0,12.8,NaN,1014.0,NaN
2022-01-04,21.6,19.6,24.0,NaN,NaN,180.0,12.4,NaN,1013.1,NaN
2022-01-05,21.3,19.0,24.0,NaN,NaN,180.0,16.6,NaN,1011.5,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-10-10,16.1,14.8,18.4,NaN,NaN,197.0,13.1,NaN,1015.2,NaN
2022-10-11,16.0,14.9,17.8,NaN,NaN,193.0,12.6,NaN,1015.6,NaN
2022-10-12,16.0,14.9,17.9,NaN,NaN,190.0,12.8,NaN,1016.1,NaN


In [4]:
## recuperación de la data de temperatura promedio, mínima y máxima
fig = px.line(df_meteostat, x=df_meteostat.index, y=["tavg", "tmin", "tmax"])

## personalización del gráfico
fig.update_layout(
    title = "Temperatura promedio, mínima, máxima (2022) - METEOSTAT"
)

## ploteo de la figura
fig.show()

## NASA POWER

In [5]:
import os, json, requests
import pandas as pd
from pandas import json_normalize

import matplotlib.pyplot as plt
import plotly.express as px

from datetime import datetime

In [6]:
## definición de parámetros
locations = [(-12.0891, -76.9988)]
start = 20220101
end = 20221231
#base_url = "https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,PS,WS10M&community=AG&longitude=0&latitude=0&start=20170101&end=20170201&format=JSON"
base_url = "https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,RH2M,PS,WS10M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude={longitude}&latitude={latitude}&start={start}&end={end}&format=JSON"

## iteración sobre la lista de coordenadas
for lat, lon in locations:
    ## petición de las coordenadas a la API
    api_request_url = base_url.format(latitude=lat, longitude=lon, start=start, end=end)
    response = requests.get(url=api_request_url, verify=True, timeout=30.00)
    print(response)

    ## extracción de la información
    content = json.loads(response.content.decode("utf-8"))
    print(content)
    filename = response.headers["content-disposition"]
    print(filename)

<Response [200]>
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-76.9988, -12.0891, 1194.76]}, 'properties': {'parameter': {'T2M': {'20220101': 19.4, '20220102': 19.54, '20220103': 19.38, '20220104': 18.89, '20220105': 18.8, '20220106': 18.99, '20220107': 19.37, '20220108': 18.75, '20220109': 18.67, '20220110': 18.97, '20220111': 19.3, '20220112': 18.72, '20220113': 18.81, '20220114': 18.42, '20220115': 18.79, '20220116': 19.22, '20220117': 20.05, '20220118': 19.73, '20220119': 19.23, '20220120': 19.62, '20220121': 19.55, '20220122': 19.37, '20220123': 19.23, '20220124': 19.2, '20220125': 19.26, '20220126': 19.08, '20220127': 18.87, '20220128': 17.29, '20220129': 18.12, '20220130': 17.42, '20220131': 18.26, '20220201': 18.05, '20220202': 18.44, '20220203': 18.65, '20220204': 18.77, '20220205': 19.73, '20220206': 19.13, '20220207': 18.15, '20220208': 18.53, '20220209': 18.13, '20220210': 18.43, '20220211': 19.0, '20220212': 18.62, '20220213': 18.88, '20220214': 19.01,

In [7]:
## descripción de los parámetros
content["parameters"]

{'T2M': {'units': 'C', 'longname': 'Temperature at 2 Meters'},
 'RH2M': {'units': '%', 'longname': 'Relative Humidity at 2 Meters'},
 'PS': {'units': 'kPa', 'longname': 'Surface Pressure'},
 'WS10M': {'units': 'm/s', 'longname': 'Wind Speed at 10 Meters'},
 'T2MDEW': {'units': 'C', 'longname': 'Dew/Frost Point at 2 Meters'},
 'T2MWET': {'units': 'C', 'longname': 'Wet Bulb Temperature at 2 Meters'},
 'TS': {'units': 'C', 'longname': 'Earth Skin Temperature'},
 'T2M_RANGE': {'units': 'C', 'longname': 'Temperature at 2 Meters Range'},
 'T2M_MAX': {'units': 'C', 'longname': 'Temperature at 2 Meters Maximum'},
 'T2M_MIN': {'units': 'C', 'longname': 'Temperature at 2 Meters Minimum'}}

In [8]:
## conversión de JSON a Pandas Dataframe
df_T2M = pd.DataFrame(content["properties"]["parameter"]["T2M"].items(), columns=['date', 'T2M'])
df_T2MDEW = pd.DataFrame(content["properties"]["parameter"]["T2MDEW"].items(), columns=['date', 'T2MDEW'])
df_T2MWET = pd.DataFrame(content["properties"]["parameter"]["T2MWET"].items(), columns=['date', 'T2MWET'])
df_TS = pd.DataFrame(content["properties"]["parameter"]["TS"].items(), columns=['date', 'TS'])
df_T2M_RANGE = pd.DataFrame(content["properties"]["parameter"]["T2M_RANGE"].items(), columns=['date', 'T2M_RANGE'])
df_T2M_MAX = pd.DataFrame(content["properties"]["parameter"]["T2M_MAX"].items(), columns=['date', 'T2M_MAX'])
df_T2M_MIN = pd.DataFrame(content["properties"]["parameter"]["T2M_MIN"].items(), columns=['date', 'T2M_MIN'])
PS = pd.DataFrame(content["properties"]["parameter"]["PS"].items(), columns=['date', 'PS'])
RH2M = pd.DataFrame(content["properties"]["parameter"]["RH2M"].items(), columns=['date', 'RH2M'])


## concatenado de Dataframes
df_nasa = pd.concat([df_T2M,
                     df_T2MDEW["T2MDEW"],
                     df_T2MWET["T2MWET"],
                     df_TS["TS"],
                     df_T2M_RANGE["T2M_RANGE"],
                     df_T2M_MAX["T2M_MAX"],
                     df_T2M_MIN["T2M_MIN"],
                     PS["PS"],
                     RH2M["RH2M"]], axis=1)

## conversión a tipo de fecha
# datetime.strptime("20220905","%Y%m%d").date()
df_nasa["date_"] = df_nasa["date"].apply(lambda x: datetime.strptime(str(x),"%Y%m%d").date())
df_nasa

,date,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,PS,RH2M,date_
0,20220101,19.40,12.23,15.81,22.38,10.32,25.29,14.98,88.67,69.06,2022-01-01
1,20220102,19.54,13.01,16.27,22.58,9.21,24.81,15.61,88.78,70.25,2022-01-02
2,20220103,19.38,14.00,16.69,22.19,7.31,23.94,16.62,88.78,74.38,2022-01-03
3,20220104,18.89,14.10,16.49,21.77,6.95,22.83,15.88,88.70,76.06,2022-01-04
4,20220105,18.80,13.70,16.26,21.49,6.68,22.62,15.94,88.58,74.69,2022-01-05
...,...,...,...,...,...,...,...,...,...,...,...
274,20221002,17.89,8.40,13.14,19.71,8.35,22.27,13.92,88.82,54.94,2022-10-02
275,20221003,17.19,9.20,13.19,19.05,7.86,21.62,13.76,88.85,60.56,2022-10-03
276,20221004,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,2022-10-04
277,20221005,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,2022-10-05


In [9]:
## recuperación de la data de temperatura promedio, mínima y máxima
fig = px.line(df_nasa.head(247), x="date_", y=["T2M", "T2M_MIN", "T2M_MAX"])

## personalización del gráfico
fig.update_layout(
    title = "Temperatura promedio, mínima, máxima (2022) - NASA POWER"
)

## ploteo de la figura
fig.show()

## Comparación de APIs

In [10]:
import plotly.graph_objects as go

In [11]:
## recuperación de la data de temperatura promedio
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_nasa["date_"].head(247), y=df_nasa["T2M"].head(247), name="NASA POWER"))
fig.add_trace(go.Scatter(x=df_meteostat.index, y=df_meteostat["tavg"], name="METEOSTAT"))

## personalización del gráfico
fig.update_layout(
    title="Temperatura Promedio (2022)", xaxis_title="date", yaxis_title="temperatura"
)

## ploteo de la figura
fig.show()

# Referencias

* https://meteostat.net/en/blog/obtain-weather-data-any-location-python [Documentación METEOSTAT]
* https://meteostat.net/en/ [Proyecto METEOSTAT]
* https://dev.meteostat.net/formats.html#time-format [Detalles de la información METEOSTAT]
* https://www.geodatos.net/coordenadas/peru/lima [Coordenadas Lima]
* https://power.larc.nasa.gov/data-access-viewer/ [Visualización de POWER]
* https://power.larc.nasa.gov/ [NASA POWER]
* https://power.larc.nasa.gov/doc [Documentación de POWER]
* https://power.larc.nasa.gov/docs/tutorials/service-data-request/api/ [Get Data from API Python]
* https://power.larc.nasa.gov/docs/services/api/temporal/daily/ [Documentación Daily]

